In [1]:
import pandas as pd
import numpy as np
import os 

# Merge (솔 코드)

In [2]:
# df = pd.read_excel('일부_취합_완료.xlsx')
# list = ['가축질병발생현황3_농림축산식품부_읍면동.xlsx', '국내발생현황_가축전염병_읍면동.xlsx', '토양최종_읍면동.xlsx', '2020년도 기준 광역지자체별 가축분뇨 발생량_읍면동.xlsx']
# for i in list :
#     df_2 = pd.read_excel(f'./{i}')
#     df = pd.merge(df, df_2, on=['시도', '시군', '읍면동'], how='outer')
# df

# 데이터 취합

In [3]:
os.listdir()

['.ipynb_checkpoints',
 '2019년 가축분뇨 농가수_시도별_v1.xlsx',
 '2019년 가축분뇨 두수_시도별_v1.xlsx',
 '2020_가축분뇨처리시설_v1.xlsx',
 '2020년도 기준 광역지자체별 가축분뇨 발생량_v1.xlsx',
 'Untitled.ipynb',
 '~$2020_가축분뇨처리시설_v1.xlsx',
 '가축질병발생현황3_농림축산식품부_v1.xlsx',
 '가축질병현황_취합본.xlsx',
 '국내발생현황_가축전염병_v1.xlsx',
 '토양최종.xlsx',
 '폭염_v1.xlsx',
 '하천수질오염데이터_v1.xlsx',
 '행정동_주소.xlsx']

In [4]:
location_df = pd.read_excel('행정동_주소.xlsx')
국내발생_df = pd.read_excel('국내발생현황_가축전염병_v1.xlsx')
가축질병발생_df = pd.read_excel('가축질병발생현황3_농림축산식품부_v1.xlsx')
hot_df = pd.read_excel('폭염_v1.xlsx')

In [5]:
분뇨처리시설_df = pd.read_excel('2020_가축분뇨처리시설_v1.xlsx')

# 1. 독립변수- 질병발생 데이터 취합

In [6]:
국내발생_df = 국내발생_df[국내발생_df['가축전염병명'] != '결핵병']
국내발생_df.head()

,시도,시군,행정동,농장명(농장주),가축전염병명,축종(품종),발생두수(마리),진단기관
21,강원도,양구군,국토정중앙면,농업회사법인 주식회사 금돈,아프리카돼지열병,돼지,10.0,강원 동물위생시험소
114,강원도,홍천군,화촌면,충성농장,아프리카돼지열병,돼지,15.0,강원 동물위생시험소
169,경상남도,김해시,한림면,세광축산,고병원성조류인플루엔자,닭,NaN,조류인플루엔자연구진단과
198,전라남도,고흥군,남양면,순수오리농장,고병원성조류인플루엔자,오리,NaN,조류인플루엔자연구진단과
212,경상남도,하동군,진교면,정진농장,고병원성조류인플루엔자,오리,NaN,조류인플루엔자연구진단과


- 발생두수(마리)는 결핵병을 제외한 결과, 몇 건 없고, '가축질병발생' 데이터에는 존재하지 않아 삭제
- 진단기관 필요없으므로 삭제

In [7]:
국내발생_df.drop(columns=['발생두수(마리)', '진단기관'], inplace=True)

In [8]:
국내발생_df.head()

,시도,시군,행정동,농장명(농장주),가축전염병명,축종(품종)
21,강원도,양구군,국토정중앙면,농업회사법인 주식회사 금돈,아프리카돼지열병,돼지
114,강원도,홍천군,화촌면,충성농장,아프리카돼지열병,돼지
169,경상남도,김해시,한림면,세광축산,고병원성조류인플루엔자,닭
198,전라남도,고흥군,남양면,순수오리농장,고병원성조류인플루엔자,오리
212,경상남도,하동군,진교면,정진농장,고병원성조류인플루엔자,오리


## 1-1. 컬럼명 통일 

In [9]:
국내발생_df.rename(columns={'행정동':'읍면동',
                        '가축전염병명':'질병명'}, inplace=True)
국내발생_df.head()

,시도,시군,읍면동,농장명(농장주),질병명,축종(품종)
21,강원도,양구군,국토정중앙면,농업회사법인 주식회사 금돈,아프리카돼지열병,돼지
114,강원도,홍천군,화촌면,충성농장,아프리카돼지열병,돼지
169,경상남도,김해시,한림면,세광축산,고병원성조류인플루엔자,닭
198,전라남도,고흥군,남양면,순수오리농장,고병원성조류인플루엔자,오리
212,경상남도,하동군,진교면,정진농장,고병원성조류인플루엔자,오리


In [10]:
국내발생_df.columns

Index(['시도', '시군', '읍면동', '농장명(농장주)', '질병명', '축종(품종)'], dtype='object')

## 1-2. concat을 위한 열 맞춤

In [11]:
가축질병발생_df['농장명(농장주)'] = np.empty(197)
가축질병발생_df['농장명(농장주)'][:] = np.nan
가축질병발생_df

C:\Users\User\AppData\Local\Temp\ipykernel_25968\1937281447.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  가축질병발생_df['농장명(농장주)'][:] = np.nan


,시도,시군,읍면동,질병명,발생일자,농장명(농장주)
0,경기도,파주시,연다산동,아프리카돼지열병,2019.09.16.,NaN
1,경기도,연천군,백학면,아프리카돼지열병,2019.09.17.,NaN
2,경기도,김포시,통진읍,아프리카돼지열병,2019.09.23.,NaN
3,경기도,파주시,적성면,아프리카돼지열병,2019.09.23.,NaN
4,인천광역시,강화군,송해면,아프리카돼지열병,2019.09.24.,NaN
...,...,...,...,...,...,...
192,전라남도,고흥군,남양면,조류인플루엔자,2022.03.02.,NaN
193,경상남도,김해시,한림면,조류인플루엔자,2022.04.07.,NaN
194,경기도,안성시,금광면,구제역,2019.01.28,NaN
195,경기도,안성시,양성면,구제역,2019.01.29,NaN


In [12]:
가축질병발생_df['축종(품종)'] = np.empty(197)
가축질병발생_df['축종(품종)'] = np.nan
가축질병발생_df.head()

,시도,시군,읍면동,질병명,발생일자,농장명(농장주),축종(품종)
0,경기도,파주시,연다산동,아프리카돼지열병,2019.09.16.,NaN,NaN
1,경기도,연천군,백학면,아프리카돼지열병,2019.09.17.,NaN,NaN
2,경기도,김포시,통진읍,아프리카돼지열병,2019.09.23.,NaN,NaN
3,경기도,파주시,적성면,아프리카돼지열병,2019.09.23.,NaN,NaN
4,인천광역시,강화군,송해면,아프리카돼지열병,2019.09.24.,NaN,NaN


In [13]:
국내발생_df['발생일자'] = np.empty(53)
국내발생_df['발생일자'] = np.nan
국내발생_df.head()

,시도,시군,읍면동,농장명(농장주),질병명,축종(품종),발생일자
21,강원도,양구군,국토정중앙면,농업회사법인 주식회사 금돈,아프리카돼지열병,돼지,NaN
114,강원도,홍천군,화촌면,충성농장,아프리카돼지열병,돼지,NaN
169,경상남도,김해시,한림면,세광축산,고병원성조류인플루엔자,닭,NaN
198,전라남도,고흥군,남양면,순수오리농장,고병원성조류인플루엔자,오리,NaN
212,경상남도,하동군,진교면,정진농장,고병원성조류인플루엔자,오리,NaN


## 1-3. 컬럼 순서 통일

In [14]:
국내발생_df.head()

,시도,시군,읍면동,농장명(농장주),질병명,축종(품종),발생일자
21,강원도,양구군,국토정중앙면,농업회사법인 주식회사 금돈,아프리카돼지열병,돼지,NaN
114,강원도,홍천군,화촌면,충성농장,아프리카돼지열병,돼지,NaN
169,경상남도,김해시,한림면,세광축산,고병원성조류인플루엔자,닭,NaN
198,전라남도,고흥군,남양면,순수오리농장,고병원성조류인플루엔자,오리,NaN
212,경상남도,하동군,진교면,정진농장,고병원성조류인플루엔자,오리,NaN


In [15]:
국내발생_df = 국내발생_df[['시도', '시군', '읍면동', '질병명', '농장명(농장주)', '축종(품종)', '발생일자']]
국내발생_df.head()

,시도,시군,읍면동,질병명,농장명(농장주),축종(품종),발생일자
21,강원도,양구군,국토정중앙면,아프리카돼지열병,농업회사법인 주식회사 금돈,돼지,NaN
114,강원도,홍천군,화촌면,아프리카돼지열병,충성농장,돼지,NaN
169,경상남도,김해시,한림면,고병원성조류인플루엔자,세광축산,닭,NaN
198,전라남도,고흥군,남양면,고병원성조류인플루엔자,순수오리농장,오리,NaN
212,경상남도,하동군,진교면,고병원성조류인플루엔자,정진농장,오리,NaN


In [16]:
가축질병발생_df

,시도,시군,읍면동,질병명,발생일자,농장명(농장주),축종(품종)
0,경기도,파주시,연다산동,아프리카돼지열병,2019.09.16.,NaN,NaN
1,경기도,연천군,백학면,아프리카돼지열병,2019.09.17.,NaN,NaN
2,경기도,김포시,통진읍,아프리카돼지열병,2019.09.23.,NaN,NaN
3,경기도,파주시,적성면,아프리카돼지열병,2019.09.23.,NaN,NaN
4,인천광역시,강화군,송해면,아프리카돼지열병,2019.09.24.,NaN,NaN
...,...,...,...,...,...,...,...
192,전라남도,고흥군,남양면,조류인플루엔자,2022.03.02.,NaN,NaN
193,경상남도,김해시,한림면,조류인플루엔자,2022.04.07.,NaN,NaN
194,경기도,안성시,금광면,구제역,2019.01.28,NaN,NaN
195,경기도,안성시,양성면,구제역,2019.01.29,NaN,NaN


In [17]:
가축질병발생_df = 가축질병발생_df[['시도', '시군', '읍면동', '질병명', '농장명(농장주)', '축종(품종)','발생일자']]
가축질병발생_df.head()

,시도,시군,읍면동,질병명,농장명(농장주),축종(품종),발생일자
0,경기도,파주시,연다산동,아프리카돼지열병,NaN,NaN,2019.09.16.
1,경기도,연천군,백학면,아프리카돼지열병,NaN,NaN,2019.09.17.
2,경기도,김포시,통진읍,아프리카돼지열병,NaN,NaN,2019.09.23.
3,경기도,파주시,적성면,아프리카돼지열병,NaN,NaN,2019.09.23.
4,인천광역시,강화군,송해면,아프리카돼지열병,NaN,NaN,2019.09.24.


In [18]:
가축질병발생_df['질병명'].unique()

array(['아프리카돼지열병', '조류인플루엔자', '구제역'], dtype=object)

In [19]:
print(len(국내발생_df))
print(len(가축질병발생_df))

53
197


## 1-4. concat

In [20]:
print(가축질병발생_df.head())
print('-'*100)
print(국내발생_df.head())

      시도   시군   읍면동       질병명  농장명(농장주)  축종(품종)         발생일자
0    경기도  파주시  연다산동  아프리카돼지열병       NaN     NaN  2019.09.16.
1    경기도  연천군   백학면  아프리카돼지열병       NaN     NaN  2019.09.17.
2    경기도  김포시   통진읍  아프리카돼지열병       NaN     NaN  2019.09.23.
3    경기도  파주시   적성면  아프리카돼지열병       NaN     NaN  2019.09.23.
4  인천광역시  강화군   송해면  아프리카돼지열병       NaN     NaN  2019.09.24.
----------------------------------------------------------------------------------------------------
       시도   시군     읍면동          질병명        농장명(농장주) 축종(품종)  발생일자
21    강원도  양구군  국토정중앙면     아프리카돼지열병  농업회사법인 주식회사 금돈     돼지   NaN
114   강원도  홍천군     화촌면     아프리카돼지열병            충성농장     돼지   NaN
169  경상남도  김해시     한림면  고병원성조류인플루엔자            세광축산      닭   NaN
198  전라남도  고흥군     남양면  고병원성조류인플루엔자          순수오리농장     오리   NaN
212  경상남도  하동군     진교면  고병원성조류인플루엔자            정진농장     오리   NaN


In [21]:
pd.concat([가축질병발생_df, 국내발생_df]).to_excel('가축질병현황_취합본.xlsx', encoding='utf-8-sig', index=False)
pd.concat([가축질병발생_df, 국내발생_df])

,시도,시군,읍면동,질병명,농장명(농장주),축종(품종),발생일자
0,경기도,파주시,연다산동,아프리카돼지열병,NaN,NaN,2019.09.16.
1,경기도,연천군,백학면,아프리카돼지열병,NaN,NaN,2019.09.17.
2,경기도,김포시,통진읍,아프리카돼지열병,NaN,NaN,2019.09.23.
3,경기도,파주시,적성면,아프리카돼지열병,NaN,NaN,2019.09.23.
4,인천광역시,강화군,송해면,아프리카돼지열병,NaN,NaN,2019.09.24.
...,...,...,...,...,...,...,...
367,충청북도,음성군,금왕읍,고병원성조류인플루엔자,일승오리농장(곽운룡),오리,NaN
369,전라남도,나주시,세지면,고병원성조류인플루엔자,한빛농장,오리,NaN
375,충청북도,음성군,금왕읍,고병원성조류인플루엔자,매봉농장,오리,NaN
376,충청북도,음성군,금왕읍,고병원성조류인플루엔자,농업회사법인 주식회사 에버그린(금왕지점),메추리,NaN


# 2. 독립변수- 폭염데이터 merge

## 2-1. 시군의 null값 탐색 (세종시일듯)

In [22]:
hot_df

,시도,시군,읍면동,폭염일수_2021,폭염일수_2020,폭염일수_2019,경도_x,위도_x
0,강원도,고성군,토성면,4.0,8.0,10.0,128.56473,38.25085
1,강원도,춘천시,NaN,21.0,7.0,18.0,127.75443,37.94738
2,강원도,철원군,갈말읍,14.0,3.0,11.0,127.30420,38.14787
3,경기도,동두천시,생연동,12.0,4.0,12.0,127.06070,37.90188
4,경기도,파주시,문산읍,13.0,2.0,10.0,126.76648,37.88589
...,...,...,...,...,...,...,...,...
91,경상남도,합천군,합천읍,17.0,14.0,27.0,128.16994,35.56505
92,경상남도,밀양시,내이동,31.0,21.0,24.0,128.74412,35.49147
93,경상남도,산청군,산청읍,12.0,9.0,19.0,127.87910,35.41300
94,경상남도,거제시,신현읍,4.0,6.0,10.0,128.60459,34.88818


In [23]:
location_df

,시도,시군구,행정동,법정동
0,서울특별시,서울특별시,서울특별시,서울특별시
1,서울특별시,종로구,종로구,종로구
2,서울특별시,종로구,청운효자동,청운동
3,서울특별시,종로구,청운효자동,신교동
4,서울특별시,종로구,청운효자동,궁정동
...,...,...,...,...
21718,제주특별자치도,서귀포시,중문동,대포동
21719,제주특별자치도,서귀포시,중문동,하원동
21720,제주특별자치도,서귀포시,예래동,색달동
21721,제주특별자치도,서귀포시,예래동,상예동


In [24]:
hot_df['시군'].isna().sum()

2

In [25]:
hot_df[hot_df['시군'].isna()]

,시도,시군,읍면동,폭염일수_2021,폭염일수_2020,폭염일수_2019,경도_x,위도_x
59,세종특별시,NaN,NaN,16.0,NaN,NaN,127.24438,36.48522
60,세종특별시,NaN,NaN,NaN,10.0,NaN,127.23500,36.48639


In [26]:
hot_df['시군'].fillna('세종특별시', inplace=True)
hot_df[hot_df['시도'] == '세종특별시']

,시도,시군,읍면동,폭염일수_2021,폭염일수_2020,폭염일수_2019,경도_x,위도_x
59,세종특별시,세종특별시,NaN,16.0,NaN,NaN,127.24438,36.48522
60,세종특별시,세종특별시,NaN,NaN,10.0,NaN,127.23500,36.48639


## 2-2. '시군' 기준으로 merge

In [27]:
hot_df

,시도,시군,읍면동,폭염일수_2021,폭염일수_2020,폭염일수_2019,경도_x,위도_x
0,강원도,고성군,토성면,4.0,8.0,10.0,128.56473,38.25085
1,강원도,춘천시,NaN,21.0,7.0,18.0,127.75443,37.94738
2,강원도,철원군,갈말읍,14.0,3.0,11.0,127.30420,38.14787
3,경기도,동두천시,생연동,12.0,4.0,12.0,127.06070,37.90188
4,경기도,파주시,문산읍,13.0,2.0,10.0,126.76648,37.88589
...,...,...,...,...,...,...,...,...
91,경상남도,합천군,합천읍,17.0,14.0,27.0,128.16994,35.56505
92,경상남도,밀양시,내이동,31.0,21.0,24.0,128.74412,35.49147
93,경상남도,산청군,산청읍,12.0,9.0,19.0,127.87910,35.41300
94,경상남도,거제시,신현읍,4.0,6.0,10.0,128.60459,34.88818


In [28]:
location_df

,시도,시군구,행정동,법정동
0,서울특별시,서울특별시,서울특별시,서울특별시
1,서울특별시,종로구,종로구,종로구
2,서울특별시,종로구,청운효자동,청운동
3,서울특별시,종로구,청운효자동,신교동
4,서울특별시,종로구,청운효자동,궁정동
...,...,...,...,...
21718,제주특별자치도,서귀포시,중문동,대포동
21719,제주특별자치도,서귀포시,중문동,하원동
21720,제주특별자치도,서귀포시,예래동,색달동
21721,제주특별자치도,서귀포시,예래동,상예동


In [29]:
location_df.rename(columns={'시군구':'시군'}, inplace=True)
location_df

,시도,시군,행정동,법정동
0,서울특별시,서울특별시,서울특별시,서울특별시
1,서울특별시,종로구,종로구,종로구
2,서울특별시,종로구,청운효자동,청운동
3,서울특별시,종로구,청운효자동,신교동
4,서울특별시,종로구,청운효자동,궁정동
...,...,...,...,...
21718,제주특별자치도,서귀포시,중문동,대포동
21719,제주특별자치도,서귀포시,중문동,하원동
21720,제주특별자치도,서귀포시,예래동,색달동
21721,제주특별자치도,서귀포시,예래동,상예동


In [30]:
df = pd.merge(location_df, hot_df, on='시군', how='left').drop(columns=['시도_y', '경도_x', '위도_x']).rename(columns={'시도_x':'시도',
                                                                                                         '읍면동':'폭염_읍면동'})

# 3. 독립변수- 분뇨처리시설 merge

## 3-1. 컬렴명 정리

In [31]:
분뇨처리시설_df

,시도,시군,읍면동,처리장명,처리공법,방류수역_천,방류수역_강,자원화 시설\n포함여부_종류,처리구분,하수처리시설
0,부산광역시,사상구,감전동,부산위생처리장,Seil-Bio공법,NaN,NaN,NaN,연계,강변하수종말처리장
1,인천광역시,서구,가좌동,가좌분뇨처리장,HBR-II,NaN,NaN,NaN,연계,가좌하수종말처리장
2,인천광역시,강화군,선원면,강화군가축분뇨공공처리시설,"BCS, YM(퇴비)",NaN,NaN,퇴비화,연계,강화하수종말처리장
3,울산광역시,울주군,온산읍,온산바이오에너지센터,혐기성소화,NaN,NaN,바이오,연계,온산하수종말처리장
4,세종특별자치시,세종특별시,부강면,등곡가축분뇨공공처리시설,BCS,NaN,금강,액비화,단독,NaN
...,...,...,...,...,...,...,...,...,...,...
103,경상남도,함양군,함양읍,함양축산폐수분뇨합병처리장,액상부식,NaN,NaN,NaN,연계,함양하수종말처리장
104,경상남도,거창군,거창읍,거창군가축분뇨공공처리장,KHTS,거창위천,NaN,NaN,단독,NaN
105,경상남도,합천군,대양면,합천군축산폐수처리장,KHTS,아천,NaN,NaN,단독,NaN
106,제주특별자치도,제주시,한림읍,제주시가축분뇨공공처리장,HYCEM,금성천,NaN,NaN,단독,NaN


In [32]:
분뇨처리시설_df.columns

Index(['시도', '시군', '읍면동', '처리장명', '처리공법', '방류수역_천', '방류수역_강',
       '자원화 시설\n포함여부_종류', '처리구분', '하수처리시설'],
      dtype='object')

In [33]:
분뇨처리시설_df.rename(columns={'자원화 시설\n포함여부_종류':'자원화 시설 포함여부_종류',
                         '읍면동':'처리시설_읍면동'}, inplace=True)
분뇨처리시설_df

,시도,시군,처리시설_읍면동,처리장명,처리공법,방류수역_천,방류수역_강,자원화 시설 포함여부_종류,처리구분,하수처리시설
0,부산광역시,사상구,감전동,부산위생처리장,Seil-Bio공법,NaN,NaN,NaN,연계,강변하수종말처리장
1,인천광역시,서구,가좌동,가좌분뇨처리장,HBR-II,NaN,NaN,NaN,연계,가좌하수종말처리장
2,인천광역시,강화군,선원면,강화군가축분뇨공공처리시설,"BCS, YM(퇴비)",NaN,NaN,퇴비화,연계,강화하수종말처리장
3,울산광역시,울주군,온산읍,온산바이오에너지센터,혐기성소화,NaN,NaN,바이오,연계,온산하수종말처리장
4,세종특별자치시,세종특별시,부강면,등곡가축분뇨공공처리시설,BCS,NaN,금강,액비화,단독,NaN
...,...,...,...,...,...,...,...,...,...,...
103,경상남도,함양군,함양읍,함양축산폐수분뇨합병처리장,액상부식,NaN,NaN,NaN,연계,함양하수종말처리장
104,경상남도,거창군,거창읍,거창군가축분뇨공공처리장,KHTS,거창위천,NaN,NaN,단독,NaN
105,경상남도,합천군,대양면,합천군축산폐수처리장,KHTS,아천,NaN,NaN,단독,NaN
106,제주특별자치도,제주시,한림읍,제주시가축분뇨공공처리장,HYCEM,금성천,NaN,NaN,단독,NaN


In [34]:
df = pd.merge(df, 분뇨처리시설_df, on='시군', how='left').drop(columns=['시도_y']).rename(columns={'시도_x':'시도'})
df.head()

,시도,시군,행정동,법정동,폭염_읍면동,폭염일수_2021,폭염일수_2020,폭염일수_2019,처리시설_읍면동,처리장명,처리공법,방류수역_천,방류수역_강,자원화 시설 포함여부_종류,처리구분,하수처리시설
0,서울특별시,서울특별시,서울특별시,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시,종로구,종로구,종로구,송월동,18.0,4.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시,종로구,청운효자동,청운동,송월동,18.0,4.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시,종로구,청운효자동,신교동,송월동,18.0,4.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시,종로구,청운효자동,궁정동,송월동,18.0,4.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
print(df.info())
print('-'*100)
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24928 entries, 0 to 24927
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   시도              24928 non-null  object 
 1   시군              24928 non-null  object 
 2   행정동             24928 non-null  object 
 3   법정동             24928 non-null  object 
 4   폭염_읍면동          12362 non-null  object 
 5   폭염일수_2021       11913 non-null  float64
 6   폭염일수_2020       11878 non-null  float64
 7   폭염일수_2019       12018 non-null  float64
 8   처리시설_읍면동        13918 non-null  object 
 9   처리장명            13918 non-null  object 
 10  처리공법            13918 non-null  object 
 11  방류수역_천          3616 non-null   object 
 12  방류수역_강          997 non-null    object 
 13  자원화 시설 포함여부_종류  3573 non-null   object 
 14  처리구분            13918 non-null  object 
 15  하수처리시설          8456 non-null   object 
dtypes: float64(3), object(13)
memory usage: 3.2+ MB
None
-----------------------

# 4. 독립변수- 가축 농가수_시도별 merge

In [36]:
os.listdir()

['.ipynb_checkpoints',
 '2019년 가축분뇨 농가수_시도별_v1.xlsx',
 '2019년 가축분뇨 두수_시도별_v1.xlsx',
 '2020_가축분뇨처리시설_v1.xlsx',
 '2020년도 기준 광역지자체별 가축분뇨 발생량_v1.xlsx',
 'Untitled.ipynb',
 '~$2020_가축분뇨처리시설_v1.xlsx',
 '가축질병발생현황3_농림축산식품부_v1.xlsx',
 '가축질병현황_취합본.xlsx',
 '국내발생현황_가축전염병_v1.xlsx',
 '토양최종.xlsx',
 '폭염_v1.xlsx',
 '하천수질오염데이터_v1.xlsx',
 '행정동_주소.xlsx']

In [37]:
농가수_df = pd.read_excel('2019년 가축분뇨 농가수_시도별_v1.xlsx')
농가수_df.head()

,시도,2019_가축분뇨_농가수_총합,한육우,젖소,말,돼지,양·사슴,닭·오리,개,타조,가금기타
0,서울특별시,10,1,1.0,2,NaN,5.0,NaN,1,NaN,NaN
1,부산광역시,72,39,3.0,3,12.0,NaN,7.0,1,NaN,7.0
2,대구광역시,380,316,17.0,3,17.0,8.0,11.0,8,NaN,NaN
3,인천광역시,231,86,15.0,7,47.0,7.0,39.0,27,NaN,3.0
4,광주광역시,221,159,9.0,4,12.0,6.0,15.0,15,NaN,1.0


## 4-1. 컬럼명 변경

In [38]:
농가수_df.columns

Index(['시도', '2019_가축분뇨_농가수_총합', '한육우', '젖소', '말', '돼지', '양·사슴', '닭·오리', '개',
       '타조', '가금기타'],
      dtype='object')

In [39]:
농가수_df.rename(columns={'한육우': '농가수_한육우', '젖소':'농가수_젖소', '말':'농가수_말', '돼지':'농가수_돼지', '양·사슴':'농가수_양·사슴', 
                       '닭·오리':'농가수_닭·오리', '개':'농가수_개','타조':'농가수_타조', '가금기타':'농가수_가금기타'}, inplace=True)
농가수_df.head()

,시도,2019_가축분뇨_농가수_총합,농가수_한육우,농가수_젖소,농가수_말,농가수_돼지,농가수_양·사슴,농가수_닭·오리,농가수_개,농가수_타조,농가수_가금기타
0,서울특별시,10,1,1.0,2,NaN,5.0,NaN,1,NaN,NaN
1,부산광역시,72,39,3.0,3,12.0,NaN,7.0,1,NaN,7.0
2,대구광역시,380,316,17.0,3,17.0,8.0,11.0,8,NaN,NaN
3,인천광역시,231,86,15.0,7,47.0,7.0,39.0,27,NaN,3.0
4,광주광역시,221,159,9.0,4,12.0,6.0,15.0,15,NaN,1.0


## 4-2. merge

In [40]:
df = pd.merge(df, 농가수_df, on='시도', how='left')
df.head()

,시도,시군,행정동,법정동,폭염_읍면동,폭염일수_2021,폭염일수_2020,폭염일수_2019,처리시설_읍면동,처리장명,...,2019_가축분뇨_농가수_총합,농가수_한육우,농가수_젖소,농가수_말,농가수_돼지,농가수_양·사슴,농가수_닭·오리,농가수_개,농가수_타조,농가수_가금기타
0,서울특별시,서울특별시,서울특별시,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,...,10,1,1.0,2,NaN,5.0,NaN,1,NaN,NaN
1,서울특별시,종로구,종로구,종로구,송월동,18.0,4.0,15.0,NaN,NaN,...,10,1,1.0,2,NaN,5.0,NaN,1,NaN,NaN
2,서울특별시,종로구,청운효자동,청운동,송월동,18.0,4.0,15.0,NaN,NaN,...,10,1,1.0,2,NaN,5.0,NaN,1,NaN,NaN
3,서울특별시,종로구,청운효자동,신교동,송월동,18.0,4.0,15.0,NaN,NaN,...,10,1,1.0,2,NaN,5.0,NaN,1,NaN,NaN
4,서울특별시,종로구,청운효자동,궁정동,송월동,18.0,4.0,15.0,NaN,NaN,...,10,1,1.0,2,NaN,5.0,NaN,1,NaN,NaN


# 5. 독립변수- 가축 두구_시도별 merge

In [41]:
os.listdir()

['.ipynb_checkpoints',
 '2019년 가축분뇨 농가수_시도별_v1.xlsx',
 '2019년 가축분뇨 두수_시도별_v1.xlsx',
 '2020_가축분뇨처리시설_v1.xlsx',
 '2020년도 기준 광역지자체별 가축분뇨 발생량_v1.xlsx',
 'Untitled.ipynb',
 '~$2020_가축분뇨처리시설_v1.xlsx',
 '가축질병발생현황3_농림축산식품부_v1.xlsx',
 '가축질병현황_취합본.xlsx',
 '국내발생현황_가축전염병_v1.xlsx',
 '토양최종.xlsx',
 '폭염_v1.xlsx',
 '하천수질오염데이터_v1.xlsx',
 '행정동_주소.xlsx']

In [42]:
두수_df = pd.read_excel('2019년 가축분뇨 두수_시도별_v1.xlsx')
두수_df.head()

,시도,2019_가축분뇨_두수_총합,한육우,젖소,말,돼지,양·사슴,닭·오리,개,타조,가금기타
0,서울특별시,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
1,부산광역시,88049,1276,168.0,55,4260.0,NaN,79500.0,50,NaN,2740.0
2,대구광역시,305200,12568,928.0,134,11533.0,206.0,277756.0,2075,NaN,NaN
3,인천광역시,565154,4939,598.0,101,3127.0,95.0,400294.0,5998,NaN,150002.0
4,광주광역시,224685,4092,738.0,60,8566.0,360.0,179200.0,1669,NaN,30000.0


## 5-1. 컬럼명 변경

In [43]:
두수_df.rename(columns={'한육우': '두수_한육우', '젖소':'두수_젖소', '말':'두수_말', '돼지':'두수_돼지', '양·사슴':'두수_양·사슴', 
                       '닭·오리':'두수_닭·오리', '개':'두수_개','타조':'두수_타조', '가금기타':'두수_가금기타'}, inplace=True)
두수_df.head()

,시도,2019_가축분뇨_두수_총합,두수_한육우,두수_젖소,두수_말,두수_돼지,두수_양·사슴,두수_닭·오리,두수_개,두수_타조,두수_가금기타
0,서울특별시,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
1,부산광역시,88049,1276,168.0,55,4260.0,NaN,79500.0,50,NaN,2740.0
2,대구광역시,305200,12568,928.0,134,11533.0,206.0,277756.0,2075,NaN,NaN
3,인천광역시,565154,4939,598.0,101,3127.0,95.0,400294.0,5998,NaN,150002.0
4,광주광역시,224685,4092,738.0,60,8566.0,360.0,179200.0,1669,NaN,30000.0


## 5-2. merge

In [44]:
df = pd.merge(df, 두수_df, on='시도', how='left')
df

,시도,시군,행정동,법정동,폭염_읍면동,폭염일수_2021,폭염일수_2020,폭염일수_2019,처리시설_읍면동,처리장명,...,2019_가축분뇨_두수_총합,두수_한육우,두수_젖소,두수_말,두수_돼지,두수_양·사슴,두수_닭·오리,두수_개,두수_타조,두수_가금기타
0,서울특별시,서울특별시,서울특별시,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
1,서울특별시,종로구,종로구,종로구,송월동,18.0,4.0,15.0,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
2,서울특별시,종로구,청운효자동,청운동,송월동,18.0,4.0,15.0,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
3,서울특별시,종로구,청운효자동,신교동,송월동,18.0,4.0,15.0,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
4,서울특별시,종로구,청운효자동,궁정동,송월동,18.0,4.0,15.0,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24923,제주특별자치도,서귀포시,예래동,색달동,서귀동,NaN,NaN,NaN,대정읍,서부가축분뇨공공처리장,...,3275460,35395,4761.0,16783,523185.0,1204.0,2571924.0,22208,NaN,100000.0
24924,제주특별자치도,서귀포시,예래동,상예동,성산읍,NaN,1.0,NaN,대정읍,서부가축분뇨공공처리장,...,3275460,35395,4761.0,16783,523185.0,1204.0,2571924.0,22208,NaN,100000.0
24925,제주특별자치도,서귀포시,예래동,상예동,서귀동,NaN,NaN,NaN,대정읍,서부가축분뇨공공처리장,...,3275460,35395,4761.0,16783,523185.0,1204.0,2571924.0,22208,NaN,100000.0
24926,제주특별자치도,서귀포시,예래동,하예동,성산읍,NaN,1.0,NaN,대정읍,서부가축분뇨공공처리장,...,3275460,35395,4761.0,16783,523185.0,1204.0,2571924.0,22208,NaN,100000.0


# 6. 종속변수- 수질오염도 merge

## 6-1. 수질오염도 오염기호 평균 구하기

In [45]:
river_df = pd.read_excel('하천수질오염데이터_v1.xlsx')
river_df.head()

,시도,시군,읍면동,측정소명,연도,총대장균군수,암모니아성 질소,질산성 질소,위도,경도
0,강원도,평창군,대관령면,송천1,2022,252.0,0.073,3.792,37.653528,128.699917
1,강원도,강릉시,왕산면,송천2,2022,NaN,NaN,NaN,37.596853,128.727392
2,강원도,강릉시,왕산면,송천3,2022,NaN,NaN,NaN,37.548150,128.713672
3,강원도,평창군,진부면,봉산천,2022,NaN,NaN,NaN,37.562842,128.668208
4,강원도,정선군,여량면,송천4,2022,109.0,0.040,2.434,37.491278,128.717667


In [46]:
lst = []
for i in np.arange(0,len(river_df)):
    mean = river_df[['총대장균군수', '암모니아성 질소', '질산성 질소']].iloc[i].mean()
    lst.append(mean)

river_df['하천오염도_평균'] = lst
river_df

,시도,시군,읍면동,측정소명,연도,총대장균군수,암모니아성 질소,질산성 질소,위도,경도,하천오염도_평균
0,강원도,평창군,대관령면,송천1,2022,252.0,0.073,3.792,37.653528,128.699917,85.288333
1,강원도,강릉시,왕산면,송천2,2022,NaN,NaN,NaN,37.596853,128.727392,NaN
2,강원도,강릉시,왕산면,송천3,2022,NaN,NaN,NaN,37.548150,128.713672,NaN
3,강원도,평창군,진부면,봉산천,2022,NaN,NaN,NaN,37.562842,128.668208,NaN
4,강원도,정선군,여량면,송천4,2022,109.0,0.040,2.434,37.491278,128.717667,37.158000
...,...,...,...,...,...,...,...,...,...,...,...
952,전라남도,신안군,압해읍,금산천,2022,8729.0,0.029,0.605,34.902131,126.321400,2909.878000
953,제주특별자치도,제주시,한림읍,옹포천,2022,9504.0,0.013,13.183,33.405639,126.260139,3172.398667
954,제주특별자치도,제주시,외도이동,외도천,2022,2164.0,0.018,2.402,33.492389,126.435222,722.140000
955,제주특별자치도,서귀포시,동홍동,동홍천,2022,3010.0,0.010,1.016,33.246750,126.571778,1003.675333


## 6-2. 수질오염도 merge

In [51]:
river_df

,시도,시군,읍면동,측정소명,연도,총대장균군수,암모니아성 질소,질산성 질소,위도,경도,하천오염도_평균
0,강원도,평창군,대관령면,송천1,2022,252.0,0.073,3.792,37.653528,128.699917,85.288333
1,강원도,강릉시,왕산면,송천2,2022,NaN,NaN,NaN,37.596853,128.727392,NaN
2,강원도,강릉시,왕산면,송천3,2022,NaN,NaN,NaN,37.548150,128.713672,NaN
3,강원도,평창군,진부면,봉산천,2022,NaN,NaN,NaN,37.562842,128.668208,NaN
4,강원도,정선군,여량면,송천4,2022,109.0,0.040,2.434,37.491278,128.717667,37.158000
...,...,...,...,...,...,...,...,...,...,...,...
952,전라남도,신안군,압해읍,금산천,2022,8729.0,0.029,0.605,34.902131,126.321400,2909.878000
953,제주특별자치도,제주시,한림읍,옹포천,2022,9504.0,0.013,13.183,33.405639,126.260139,3172.398667
954,제주특별자치도,제주시,외도이동,외도천,2022,2164.0,0.018,2.402,33.492389,126.435222,722.140000
955,제주특별자치도,서귀포시,동홍동,동홍천,2022,3010.0,0.010,1.016,33.246750,126.571778,1003.675333


In [53]:
df

,시도,시군,행정동,법정동,폭염_읍면동,폭염일수_2021,폭염일수_2020,폭염일수_2019,처리시설_읍면동,처리장명,...,2019_가축분뇨_두수_총합,두수_한육우,두수_젖소,두수_말,두수_돼지,두수_양·사슴,두수_닭·오리,두수_개,두수_타조,두수_가금기타
0,서울특별시,서울특별시,서울특별시,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
1,서울특별시,종로구,종로구,종로구,송월동,18.0,4.0,15.0,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
2,서울특별시,종로구,청운효자동,청운동,송월동,18.0,4.0,15.0,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
3,서울특별시,종로구,청운효자동,신교동,송월동,18.0,4.0,15.0,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
4,서울특별시,종로구,청운효자동,궁정동,송월동,18.0,4.0,15.0,NaN,NaN,...,213,97,15.0,14,NaN,58.0,NaN,29,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24923,제주특별자치도,서귀포시,예래동,색달동,서귀동,NaN,NaN,NaN,대정읍,서부가축분뇨공공처리장,...,3275460,35395,4761.0,16783,523185.0,1204.0,2571924.0,22208,NaN,100000.0
24924,제주특별자치도,서귀포시,예래동,상예동,성산읍,NaN,1.0,NaN,대정읍,서부가축분뇨공공처리장,...,3275460,35395,4761.0,16783,523185.0,1204.0,2571924.0,22208,NaN,100000.0
24925,제주특별자치도,서귀포시,예래동,상예동,서귀동,NaN,NaN,NaN,대정읍,서부가축분뇨공공처리장,...,3275460,35395,4761.0,16783,523185.0,1204.0,2571924.0,22208,NaN,100000.0
24926,제주특별자치도,서귀포시,예래동,하예동,성산읍,NaN,1.0,NaN,대정읍,서부가축분뇨공공처리장,...,3275460,35395,4761.0,16783,523185.0,1204.0,2571924.0,22208,NaN,100000.0


In [48]:
df = pd.merge(df, river_df[['시도', '시군', '읍면동','하천오염도_평균']], on='시도', how='left')
df

In [49]:
# df = df.drop(columns=['시군_y', '읍면동']).rename(columns={'시군_x':'시군'})
# df.head()

In [50]:
# df[['하천오염도_평균', '시도', '시군', '행정동', '법정동', '폭염_읍면동', '폭염일수_2021', '폭염일수_2020',
#        '폭염일수_2019', '처리시설_읍면동', '처리장명', '처리공법', '방류수역_천', '방류수역_강',
#        '자원화 시설 포함여부_종류', '처리구분', '하수처리시설', '2019_가축분뇨_농가수_총합', '농가수_한육우',
#        '농가수_젖소', '농가수_말', '농가수_돼지', '농가수_양·사슴', '농가수_닭·오리', '농가수_개', '농가수_타조',
#        '농가수_가금기타', '2019_가축분뇨_두수_총합', '두수_한육우', '두수_젖소', '두수_말', '두수_돼지',
#        '두수_양·사슴', '두수_닭·오리', '두수_개', '두수_타조', '두수_가금기타']]